In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .appName("PySpark SQL")\
    .getOrCreate()
    
spark

In [3]:
socketDF = spark.readStream.format("socket").option("host", "seppe.net").option("port", 7777).load()

socketDF.printSchema()

root
 |-- value: string (nullable = true)



In [4]:
query = socketDF.writeStream.format("memory") \
    .trigger(processingTime='2 seconds') \
    .queryName("my_query") \
    .start()

In [9]:
from pyspark.sql import functions

df = spark.table("my_query")
split_col = functions.split(df['value'], '\t')
df = df.withColumn('subreddit', split_col.getItem(0))
df = df.withColumn('title', split_col.getItem(1))
df = df.select(['subreddit', 'title'])

df.show()

+---------+--------------------+
|subreddit|               title|
+---------+--------------------+
|askreddit|What is your most...|
|askreddit|What are you than...|
|askreddit|What is more wond...|
|askreddit|Actors/Actresses ...|
|askreddit|What do you put i...|
|askreddit|If you could impl...|
|askreddit|How do you stop y...|
|askreddit|As international ...|
|askreddit|What have you giv...|
|askreddit|What's a funny en...|
|askreddit|What's the weirde...|
|askreddit|For people who re...|
|askreddit|What is your next...|
|askreddit|Reddit, of what f...|
|askreddit|Why does a surpri...|
|askreddit|Retail workers of...|
|askreddit|What is the weird...|
|askreddit|Where are you fro...|
|askreddit|Who is the cowork...|
|askreddit|Why does Bank of ...|
+---------+--------------------+
only showing top 20 rows



In [10]:
query.stop()